# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
pd.options.display.max_columns = 1000
import psycopg2
from sql_queries import demographic_insert, temp_insert, i94_insert, visa_insert

### Step 1: Scope the Project and Gather Data

#### Scope 

Say you wish to have a data source keeping track of immigration statistics on a monthly basis. However you may also wish to examine any underlying trends and for this reason you wish to be able to extraopolate data on the cities they are immigrating to or if climate and temperature has any effect. This data could be a monthly updated dashboard allowing any changes etc to be discovered and analysed. It will be aggregated from a number of sources and combined in one data model. The process of it's creation and what it includes is described below.

#### Describe and Gather Data 
Four datasets are combined together in this data model to give an overall comprehensive view of the immigration statistics.

- I94 Immigration statistics : From the US National Trade and Tourism department collating details of immigrant arrivals in the U.S. In this use case we believe that the data will form part of a dashboard which is updated on a monthly basis so we will subset the data to April 2016.
- Demographic data : From the US Census Bureau's 2015 American Community Survey, containing information about the demographics of all US cities and census-designated places with a population greater or equal to 65,000. Therefore we believe the data is timely enough to be relevant to the immigration data.
- Temperature Data : Data sourced from Kaggle, repackaged from Berkeley Earth which looks at global temoperature on a daily basis at a country state and city level.
- State Code data : Sourced from https://scottontechnology.com/list-of-50-us-states-in-excel/#:~:text=Downloadable%20Excel%20%28.csv%29%20lists%20of%20the%2050,US%20State%20names%2C%20abbreviations%2C%20and%20AP%20style%20abbreviations. containing all of the stae names and their relevant state codes.

**READ IN DATA**

In [3]:
# Read in the data here
demographic_df = pd.read_csv('us-cities-demographics.csv', delimiter = ';')
states_df = pd.read_csv('50_us_states_all_data.csv')
temp_df = pd.read_csv('GlobalLandTemperaturesByState.csv')

demographic_df.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


Read in sample of the immigration data

In [3]:
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
immigrant_df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")


In [5]:
immigrant_df.columns

Index(['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate',
       'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count',
       'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu',
       'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline',
       'admnum', 'fltno', 'visatype'],
      dtype='object')

## Step 2: Explore and Assess Data

### states data

In [6]:
# Performing cleaning tasks here
states_df.head()


,ALABAMA,Alabama,AL,Ala.
0,ALASKA,Alaska,AK,Alaska
1,ARIZONA,Arizona,AZ,Ariz.
2,ARKANSAS,Arkansas,AR,Ark.
3,CALIFORNIA,California,CA,Calif.
4,COLORADO,Colorado,CO,Colo.


First row is being used a column titles. TO fix, we can overwrite file

In [5]:
states_df = pd.read_csv('50_us_states_all_data.csv', header= None, names= ['State_name', 'State_name_lower', 'Code', "Short_name"])
states_df.head()

,State_name,State_name_lower,Code,Short_name
0,ALABAMA,Alabama,AL,Ala.
1,ALASKA,Alaska,AK,Alaska
2,ARIZONA,Arizona,AZ,Ariz.
3,ARKANSAS,Arkansas,AR,Ark.
4,CALIFORNIA,California,CA,Calif.


Check data shape

In [6]:
states_df.shape

(50, 4)

50 states as expected.

### Demographic data

In [7]:
pd.set_option('display.float_format', '{:.4f}'.format)
demographic_df.describe()


,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,Count
count,2891.0000,2888.0000,2888.0000,2891.0000,2878.0000,2878.0000,2875.0000,2891.0000
mean,35.4949,97328.4262,101769.6309,198966.7793,9367.8325,40653.5987,2.7425,48963.7745
std,4.4016,216299.9369,231564.5726,447555.9296,13211.2199,155749.1037,0.4333,144385.5886
min,22.9000,29281.0000,27348.0000,63215.0000,416.0000,861.0000,2.0000,98.0000
25%,32.8000,39289.0000,41227.0000,80429.0000,3739.0000,9224.0000,2.4300,3435.0000
50%,35.3000,52341.0000,53809.0000,106782.0000,5397.0000,18822.0000,2.6500,13780.0000
75%,38.0000,86641.7500,89604.0000,175232.0000,9368.0000,33971.7500,2.9500,54447.0000
max,70.5000,4081698.0000,4468707.0000,8550405.0000,156961.0000,3212500.0000,4.9800,3835726.0000


Nothing seems off about this to any large extent (no high amount of nulls and all data seems appropriate). However we want to make the city and state the unique key so we need to combine rows about same state talking about different races by creating binary columns for each race and a corresponding count for each race. 

In [8]:
df_dummies = pd.get_dummies(demographic_df['Race'], prefix= 'Race')
demographic_df = pd.concat([demographic_df, df_dummies], axis =1)
for race in demographic_df['Race'].unique():
    demographic_df[f'count_{race.lower()}'] = demographic_df['Count'].where(demographic_df['Race'] == race, 0)




In [12]:
demographic_df = demographic_df.groupby(['City', 'State']).agg({
    'Median Age': 'first',  # Keep the first value (assuming it's the same)
    'Male Population': 'first',  # Sum male population
    'Female Population': 'first',  # Sum female population
    'Total Population': 'first',  # Sum total population
    'Number of Veterans': 'first',  # Sum number of veterans
    'Foreign-born': 'first',  # Sum foreign-born
    'Average Household Size': 'first',  # Keep the first value
    'State Code': 'first',
    'Race_American Indian and Alaska Native': 'sum',
    'Race_Asian': 'sum',
    'Race_Black or African-American': 'sum',
    'Race_Hispanic or Latino':'sum',
    'Race_White':'sum',
    'count_american indian and alaska native':'sum',
    'count_asian':'sum',
    'count_black or african-american': 'sum',
    'count_hispanic or latino': 'sum',
    'count_white': 'sum'

}).reset_index()


In [13]:
demographic_df.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race_American Indian and Alaska Native,Race_Asian,Race_Black or African-American,Race_Hispanic or Latino,Race_White,count_american indian and alaska native,count_asian,count_black or african-american,count_hispanic or latino,count_white
0,Abilene,Texas,31.3000,65212.0000,60664.0000,125876,9367.0000,8129.0000,2.6400,TX,1,1,1,1,1,1813,2929,14449,33222,95487
1,Akron,Ohio,38.1000,96886.0000,100667.0000,197553,12878.0000,10024.0000,2.2400,OH,1,1,1,1,1,1845,9033,66551,3684,129192
2,Alafaya,Florida,33.5000,39504.0000,45760.0000,85264,4176.0000,15842.0000,2.9400,FL,0,1,1,1,1,0,10336,6577,34897,63666
3,Alameda,California,41.4000,37747.0000,40867.0000,78614,4504.0000,18841.0000,2.5200,CA,1,1,1,1,1,1329,27984,7364,8265,44232
4,Albany,Georgia,33.3000,31695.0000,39414.0000,71109,5409.0000,861.0000,2.3800,GA,1,1,1,1,1,445,650,53440,1783,17160


In [14]:
demographic_df.to_csv('demo_df_cleaned.csv')

### temperature dataframe

In [15]:
temp_df.describe()

,AverageTemperature,AverageTemperatureUncertainty
count,620027.0000,620027.0000
mean,8.9931,1.2876
std,13.7722,1.3604
min,-45.3890,0.0360
25%,-0.6930,0.3160
50%,11.1990,0.6560
75%,19.8990,1.8500
max,36.3390,12.6460


no large amounts of zero. However one thing we want to is filter it to just the USA and create a monthly average 

In [16]:
temp_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
0,1855-05-01,25.5440,1.1710,Acre,Brazil
1,1855-06-01,24.2280,1.1030,Acre,Brazil
2,1855-07-01,24.3710,1.0440,Acre,Brazil
3,1855-08-01,25.4270,1.0730,Acre,Brazil
4,1855-09-01,25.6750,1.0140,Acre,Brazil


In [17]:
temp_df['Country'].value_counts()

Russia           254972
United States    149745
India             86664
China             68506
Canada            35358
Brazil            34328
Australia         16102
Name: Country, dtype: int64

In [18]:
temp_df_us = temp_df[temp_df['Country']== 'United States']
print(temp_df_us)

                dt  AverageTemperature  AverageTemperatureUncertainty  \
7458    1743-11-01             10.7220                         2.8980   
7459    1743-12-01                 nan                            nan   
7460    1744-01-01                 nan                            nan   
7461    1744-02-01                 nan                            nan   
7462    1744-03-01                 nan                            nan   
7463    1744-04-01             19.0750                         2.9020   
7464    1744-05-01             21.1970                         2.8440   
7465    1744-06-01             25.2900                         2.8790   
7466    1744-07-01             26.4200                         2.8410   
7467    1744-08-01                 nan                            nan   
7468    1744-09-01             21.7350                         2.8660   
7469    1744-10-01             15.6300                         2.8720   
7470    1744-11-01             11.1980             

want to group by av temp so remove any zeroes.

In [19]:
temp_df_us = temp_df_us.dropna(subset=['AverageTemperature'])

In [20]:
temp_df_us['dt'] = pd.to_datetime(temp_df_us['dt'])
temp_df_us['Year'] = temp_df_us['dt'].dt.year
temp_df_us['Month'] = temp_df_us['dt'].dt.month
monthly_avg = temp_df_us.groupby(['Year', 'Month', 'State'])['AverageTemperature','AverageTemperatureUncertainty'].mean().reset_index()

# Display the result
print(monthly_avg)

        Year  Month                 State  AverageTemperature  \
0       1743     11               Alabama             10.7220   
1       1743     11           Connecticut              3.8180   
2       1743     11              Delaware              6.1270   
3       1743     11  District Of Columbia              5.3390   
4       1743     11               Florida             17.6940   
5       1743     11       Georgia (State)             11.5140   
6       1743     11              Illinois              3.9260   
7       1743     11               Indiana              4.0450   
8       1743     11                  Iowa              1.2500   
9       1743     11              Kentucky              6.1310   
10      1743     11                 Maine             -1.2220   
11      1743     11              Maryland              5.9530   
12      1743     11         Massachusetts              1.8910   
13      1743     11              Michigan              0.7530   
14      1743     11      

We would like to subset the data so it only consists of april 2016 like our i94 data. However our data only extends to 2013. As this is only a POC we will use April 2013 data as a stand in for 2106 as it would be most similar from our existing data.

In [21]:
temp_data = temp_df_us[(temp_df_us['Year']== 2013)& (temp_df_us['Month']==4)]

In [22]:
temp_data.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country,Year,Month
10691,2013-04-01,17.0670,0.2210,Alabama,United States,2013,4
15101,2013-04-01,-9.2080,0.3920,Alaska,United States,2013,4
34147,2013-04-01,15.1690,0.2790,Arizona,United States,2013,4
37214,2013-04-01,14.9220,0.1450,Arkansas,United States,2013,4
73029,2013-04-01,14.7070,0.2830,California,United States,2013,4


In [24]:
temp_data.drop('dt', axis = 1, inplace= True)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [25]:
temp_data = temp_data.reset_index()

Create an id column to help build model.

In [11]:
temp_data.rename(columns = {'index':'temp_id'}, inplace = True)

NameError: name 'temp_data' is not defined

In [12]:
temp_data.to_csv('temp_data_cleaned.csv')

NameError: name 'temp_data' is not defined

### Immigration data

In [28]:
immigrant_df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0000,2016.0000,4.0000,692.0000,692.0000,XXX,20573.0000,nan,NaN,nan,37.0000,2.0000,1.0000,NaN,NaN,NaN,T,NaN,U,NaN,1979.0000,10282016,NaN,NaN,NaN,1897628485.0000,NaN,B2
1,7.0000,2016.0000,4.0000,254.0000,276.0000,ATL,20551.0000,1.0000,AL,nan,25.0000,3.0000,1.0000,20130811,SEO,NaN,G,NaN,Y,NaN,1991.0000,D/S,M,NaN,NaN,3736796330.0000,00296,F1
2,15.0000,2016.0000,4.0000,101.0000,101.0000,WAS,20545.0000,1.0000,MI,20691.0000,55.0000,2.0000,1.0000,20160401,NaN,NaN,T,O,NaN,M,1961.0000,09302016,M,NaN,OS,666643185.0000,93,B2
3,16.0000,2016.0000,4.0000,101.0000,101.0000,NYC,20545.0000,1.0000,MA,20567.0000,28.0000,2.0000,1.0000,20160401,NaN,NaN,O,O,NaN,M,1988.0000,09302016,NaN,NaN,AA,92468461330.0000,00199,B2
4,17.0000,2016.0000,4.0000,101.0000,101.0000,NYC,20545.0000,1.0000,MA,20567.0000,4.0000,2.0000,1.0000,20160401,NaN,NaN,O,O,NaN,M,2012.0000,09302016,NaN,NaN,AA,92468463130.0000,00199,B2


In [29]:
immigrant_df.describe()

,cicid,i94yr,i94mon,i94cit,i94res,arrdate,i94mode,depdate,i94bir,i94visa,count,biryear,admnum
count,3096313.0000,3096313.0000,3096313.0000,3096313.0000,3096313.0000,3096313.0000,3096074.0000,2953856.0000,3095511.0000,3096313.0000,3096313.0000,3095511.0000,3096313.0000
mean,3078651.8791,2016.0000,4.0000,304.9069,303.2838,20559.8485,1.0737,20573.9528,41.7676,1.8454,1.0000,1974.2324,70828850111.1120
std,1763278.0997,0.0000,0.0000,210.0269,208.5832,8.7773,0.5159,29.3570,17.4203,0.3984,0.0000,17.4203,22154415947.5576
min,6.0000,2016.0000,4.0000,101.0000,101.0000,20545.0000,1.0000,15176.0000,-3.0000,1.0000,1.0000,1902.0000,0.0000
25%,1577790.0000,2016.0000,4.0000,135.0000,131.0000,20552.0000,1.0000,20561.0000,30.0000,2.0000,1.0000,1962.0000,56035228433.0000
50%,3103507.0000,2016.0000,4.0000,213.0000,213.0000,20560.0000,1.0000,20570.0000,41.0000,2.0000,1.0000,1975.0000,59360939033.0000
75%,4654341.0000,2016.0000,4.0000,512.0000,504.0000,20567.0000,1.0000,20579.0000,54.0000,2.0000,1.0000,1986.0000,93509869930.0000
max,6102785.0000,2016.0000,4.0000,999.0000,760.0000,20574.0000,9.0000,45427.0000,114.0000,3.0000,1.0000,2019.0000,99915565930.0000


Again, no vast amounts of missing dats. We want to add a state full name column to allow us to match our other datasets to the state code in the i94addr table. We will merge this with state data to add the full name column.  

In [30]:
states_df.head()

,State_name,State_name_lower,Code,Short_name
0,ALABAMA,Alabama,AL,Ala.
1,ALASKA,Alaska,AK,Alaska
2,ARIZONA,Arizona,AZ,Ariz.
3,ARKANSAS,Arkansas,AR,Ark.
4,CALIFORNIA,California,CA,Calif.


In [31]:
states_df= states_df.drop(['State_name', 'Short_name'], axis=1)

In [32]:
immigrant_merged = pd.merge(immigrant_df, states_df, left_on = 'i94addr',right_on = 'Code', how='left')

In [33]:
immigrant_merged.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype,State_name_lower,Code
0,6.0000,2016.0000,4.0000,692.0000,692.0000,XXX,20573.0000,nan,NaN,nan,37.0000,2.0000,1.0000,NaN,NaN,NaN,T,NaN,U,NaN,1979.0000,10282016,NaN,NaN,NaN,1897628485.0000,NaN,B2,NaN,NaN
1,7.0000,2016.0000,4.0000,254.0000,276.0000,ATL,20551.0000,1.0000,AL,nan,25.0000,3.0000,1.0000,20130811,SEO,NaN,G,NaN,Y,NaN,1991.0000,D/S,M,NaN,NaN,3736796330.0000,00296,F1,Alabama,AL
2,15.0000,2016.0000,4.0000,101.0000,101.0000,WAS,20545.0000,1.0000,MI,20691.0000,55.0000,2.0000,1.0000,20160401,NaN,NaN,T,O,NaN,M,1961.0000,09302016,M,NaN,OS,666643185.0000,93,B2,Michigan,MI
3,16.0000,2016.0000,4.0000,101.0000,101.0000,NYC,20545.0000,1.0000,MA,20567.0000,28.0000,2.0000,1.0000,20160401,NaN,NaN,O,O,NaN,M,1988.0000,09302016,NaN,NaN,AA,92468461330.0000,00199,B2,Massachusetts,MA
4,17.0000,2016.0000,4.0000,101.0000,101.0000,NYC,20545.0000,1.0000,MA,20567.0000,4.0000,2.0000,1.0000,20160401,NaN,NaN,O,O,NaN,M,2012.0000,09302016,NaN,NaN,AA,92468463130.0000,00199,B2,Massachusetts,MA


In [34]:
immigrant_merged.drop('Code', axis =1, inplace= True)

In [35]:
immigrant_merged.head(5)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype,State_name_lower
0,6.0000,2016.0000,4.0000,692.0000,692.0000,XXX,20573.0000,nan,NaN,nan,37.0000,2.0000,1.0000,NaN,NaN,NaN,T,NaN,U,NaN,1979.0000,10282016,NaN,NaN,NaN,1897628485.0000,NaN,B2,NaN
1,7.0000,2016.0000,4.0000,254.0000,276.0000,ATL,20551.0000,1.0000,AL,nan,25.0000,3.0000,1.0000,20130811,SEO,NaN,G,NaN,Y,NaN,1991.0000,D/S,M,NaN,NaN,3736796330.0000,00296,F1,Alabama
2,15.0000,2016.0000,4.0000,101.0000,101.0000,WAS,20545.0000,1.0000,MI,20691.0000,55.0000,2.0000,1.0000,20160401,NaN,NaN,T,O,NaN,M,1961.0000,09302016,M,NaN,OS,666643185.0000,93,B2,Michigan
3,16.0000,2016.0000,4.0000,101.0000,101.0000,NYC,20545.0000,1.0000,MA,20567.0000,28.0000,2.0000,1.0000,20160401,NaN,NaN,O,O,NaN,M,1988.0000,09302016,NaN,NaN,AA,92468461330.0000,00199,B2,Massachusetts
4,17.0000,2016.0000,4.0000,101.0000,101.0000,NYC,20545.0000,1.0000,MA,20567.0000,4.0000,2.0000,1.0000,20160401,NaN,NaN,O,O,NaN,M,2012.0000,09302016,NaN,NaN,AA,92468463130.0000,00199,B2,Massachusetts


In [36]:
immigrant_merged.to_csv('immigrant_df_cleaned.csv')

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model



Due to the structure of the data and to allow for simplicty of querying the data from less technical user, a star table is used to model the data. The immigrant data is chosen as the central fact table, and demographic, visa, and temperature data are used as enriching dimension tables. 

**Tables** 


|Table_Name|Columns|Table_type|Info|
|---|---|---|---|
|Immigrants_df|**cicid**, i94yr, i94mon, i94cit, i94res, i94port, arrdate, i94mode, i94addr, depdate, i94bir, i94visa, count, dtatdfile, occup, entdepa, entdepu, matflag, biryear, dtaddto, gender, insum, airline, admnum, fltno, State_name_lower, temp_id| Fact Table| Information on immigration details |
|Visa_df| **i94visa**, visatype| Dimension table| Information on visas and their corresponding type|
|Temperatures_df| **temp_id**, Average_temperature, Average_temperature_uncertainty, State, Country, Year, Month| Dimension Table| Information around the average temperature in April 2013 for U.S states| 
|Demographics_df| **City, State**,	Median Age,	Male Population, Female Population, Total Population, Number of Veterans, Foreign-born, Average Household Size,	State Code, Race_American Indian and Alaska Native, Race_Asian, Race_Black or African-American, Race_Hispanic or Latino, 	Race_White,	count_hispanic or latino, count_white, count_asian,	count_black or african-american,	count_american indian and alaska native| Dimension table| Information on demogrpahics of US cities|

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

1. Create Tables using Create_Tables.py
2. Join temperature data to immigrant data using temp_id
3. Insert data

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [2]:
# Write code here
# After running create_tables.py, insert the data into the database
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
demo_clean = pd.read_csv('demo_df_cleaned.csv')
temp_clean = pd.read_csv('temp_data_cleaned.csv')
immigrant_clean = pd.read_csv('immigrant_df_cleaned.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
demo_clean.drop('Unnamed: 0', axis =1, inplace=True)

In [5]:
demo_clean.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race_American Indian and Alaska Native,Race_Asian,Race_Black or African-American,Race_Hispanic or Latino,Race_White,count_american indian and alaska native,count_asian,count_black or african-american,count_hispanic or latino,count_white
0,Abilene,Texas,31.3,65212.0,60664.0,125876,9367.0,8129.0,2.64,TX,1,1,1,1,1,1813,2929,14449,33222,95487
1,Akron,Ohio,38.1,96886.0,100667.0,197553,12878.0,10024.0,2.24,OH,1,1,1,1,1,1845,9033,66551,3684,129192
2,Alafaya,Florida,33.5,39504.0,45760.0,85264,4176.0,15842.0,2.94,FL,0,1,1,1,1,0,10336,6577,34897,63666
3,Alameda,California,41.4,37747.0,40867.0,78614,4504.0,18841.0,2.52,CA,1,1,1,1,1,1329,27984,7364,8265,44232
4,Albany,Georgia,33.3,31695.0,39414.0,71109,5409.0,861.0,2.38,GA,1,1,1,1,1,445,650,53440,1783,17160


In [6]:
temp_clean.drop('Unnamed: 0', axis =1, inplace= True)

In [7]:
df_immigrant = immigrant_clean.merge(temp_clean, left_on="State_name_lower", right_on="State")

In [8]:
df_immigrant.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype,State_name_lower,temp_id,AverageTemperature,AverageTemperatureUncertainty,State,Country,Year,Month
0,1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,1.0,20130811.0,SEO,NaN,G,NaN,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1,Alabama,10691,17.067,0.221,Alabama,United States,2013,4
1,39,67.0,2016.0,4.0,103.0,103.0,ATL,20545.0,1.0,AL,20580.0,64.0,2.0,1.0,20160401.0,NaN,NaN,O,O,NaN,M,1952.0,06292016,NaN,NaN,DL,5.543515e+10,00131,WT,Alabama,10691,17.067,0.221,Alabama,United States,2013,4
2,661,779.0,2016.0,4.0,104.0,104.0,ATL,20545.0,1.0,AL,20562.0,74.0,2.0,1.0,20160401.0,NaN,NaN,G,O,NaN,M,1942.0,06292016,F,NaN,DL,5.541707e+10,00083,WT,Alabama,10691,17.067,0.221,Alabama,United States,2013,4
3,2166,2500.0,2016.0,4.0,107.0,107.0,MIA,20545.0,1.0,AL,20565.0,36.0,1.0,1.0,20160401.0,MLN,NaN,G,O,NaN,M,1980.0,09302016,F,NaN,AA,9.248440e+10,00207,B1,Alabama,10691,17.067,0.221,Alabama,United States,2013,4
4,2423,2825.0,2016.0,4.0,108.0,108.0,ATL,20545.0,1.0,AL,20546.0,45.0,1.0,1.0,20160401.0,NaN,NaN,O,I,NaN,M,1971.0,06292016,NaN,NaN,KL,5.546117e+10,00621,WB,Alabama,10691,17.067,0.221,Alabama,United States,2013,4


In [9]:
df_immigrant.drop(['Unnamed: 0', 'AverageTemperature', 'AverageTemperatureUncertainty', 'State', 'Country', 'Year', 'Month'], axis= 1, inplace= True)

In [10]:
df_immigrant.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype,State_name_lower,temp_id
0,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,1.0,20130811.0,SEO,NaN,G,NaN,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1,Alabama,10691
1,67.0,2016.0,4.0,103.0,103.0,ATL,20545.0,1.0,AL,20580.0,64.0,2.0,1.0,20160401.0,NaN,NaN,O,O,NaN,M,1952.0,06292016,NaN,NaN,DL,5.543515e+10,00131,WT,Alabama,10691
2,779.0,2016.0,4.0,104.0,104.0,ATL,20545.0,1.0,AL,20562.0,74.0,2.0,1.0,20160401.0,NaN,NaN,G,O,NaN,M,1942.0,06292016,F,NaN,DL,5.541707e+10,00083,WT,Alabama,10691
3,2500.0,2016.0,4.0,107.0,107.0,MIA,20545.0,1.0,AL,20565.0,36.0,1.0,1.0,20160401.0,MLN,NaN,G,O,NaN,M,1980.0,09302016,F,NaN,AA,9.248440e+10,00207,B1,Alabama,10691
4,2825.0,2016.0,4.0,108.0,108.0,ATL,20545.0,1.0,AL,20546.0,45.0,1.0,1.0,20160401.0,NaN,NaN,O,I,NaN,M,1971.0,06292016,NaN,NaN,KL,5.546117e+10,00621,WB,Alabama,10691


In [11]:
visa_df = df_immigrant[ ['i94visa', 'visatype']]

In [12]:

visa_df= visa_df.reset_index()
visa_df.rename(columns={'index': 'visa_id'}, inplace=True)


In [13]:
df_immigrant.drop('visatype', axis= 1, inplace= True)

In [14]:
for index, row in demo_clean.iterrows():

    cur.execute(demographic_insert, list(row.values))
    conn.commit()


In [15]:

for index, row in visa_df.iterrows():
  


    cur.execute(visa_insert, list(row.values))
    conn.commit()


In [16]:
for index, row in temp_clean.iterrows():
    cur.execute(temp_insert, list(row.values))
    conn.commit()

In [17]:

for index, row in df_immigrant.iterrows():
    cur.execute(i94_insert, list(row.values))
    conn.commit()

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [18]:
# Perform quality checks here

cur.execute("SELECT COUNT(*) FROM demo")
conn.commit()
if cur.rowcount < 1:
    print("No data found in table demo")
    
cur.execute("SELECT COUNT(*) FROM immigrant")
conn.commit()
if cur.rowcount < 1:
    print("No data found in table immigrant")
    
cur.execute("SELECT COUNT(*) FROM visa")
conn.commit()
if cur.rowcount < 1:
    print("No data found in table visa")
    
cur.execute("SELECT COUNT(*) FROM temp")
conn.commit()
if cur.rowcount < 1:
    print("No data found in table temp")

In [ ]:
cur.close()
conn.close()

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

**Data Dictionary**

|immigrant_fact|type|desc|
|---|---|---|
|cicid| FLOAT| Imiigrant_id|
|i94yr|FLOAT| Year of arrival|
|i94mon|FLOAT|Month|
|i94cit|FLOAT| Citizen Country|
|i94res|FLOAT| Residence Country|#
|i94port|VARCHAR|Port code of arrival port|
|arrdate|FLOAT|Arrival date|
|i94mode|FLOAT| Mode of transfer|
|i94arr|VARCHAR| State of arrival|
|depdate|FLOAT| departure date|
|i94bir|FLOAT| birthdate|
|i94visa|FLOAT|Visa type|
|count| FLOAT| Number of people|
|dtatdfile|FLOAT| Date of file|
|visa_post|VARCHAR|Visa Post State|
|occup| VARCHAR| Occupation|
|entdepa|VARCHAR| Arrival Flag|
|entdepd| VARCHAR| Departure Flag|
|entdepu|VARCHAR| Update Flag|
|MATFLAG| VARCHAR| Match Flag|
|biryear| Float| Birth Year|
|dtaddto| VARCHAR| Visa Valid Till|
|gender| VARCHAR| Gender|
|insum| FLOAT| INS number|
|airline|VARCHAR| Airline|
|admnum| FLOAT| Adminsitration Numbwer|
|fltno| INT| flighht number |
| State Name Lower| VARCHAR| state name in full|
|temp_id_full| INT| Temperature id|

|temp_dim| Type| Description|
|---|---|---|
|temp_id| INT| id column|
|Average_temperature| FLOAT| Average temperature for the month|
|Average_uncertainty| FLOAT| Average Uncertainity for that temperature|
|State| VARCHAR| U.S State|
|Country| VARCHAR| Country|
|Year| INT| Year data is taken from|
|Month| INT| Month data is taken from|

|demo_dim|Type|Description|
|---|---|---|
|City|VARCHAR|City being described|
|State|VARCHAR|State city is in|
|Median Age|FLOAT| Median age|
|Male Population| FLOAT| Number of males|
|Female Population|FLOAT| Number of females|
|Total Population|FLOAT| Total number of people|
|Number of Veterans| FLOAT| Number in population who are  veterans|
|Foreign-born|FLOAT| Number in population who are born abroad|
|Average Household Size|FLOAT| Average Household size|
|State Code| VARCHAR| 2 letter code for US state|
|Race_American Indian and Alaska Native| INT| Indicator if name race is large proportion|
|Race_Asian|INT| Indicator if name race is large proportion|
|Race_Black or African-American| INT|Indicator if name race is large proportion|
|Race_Hispanic or Latino|INT| Indicator if name race is large proportion|
|Race_White| INT| Indicator if name race is large proportion|
|count_hispanic or latino|INT| number of race|
|count_white|INT| number of race|
|count_asian|INT| number of race|
|count_black or african-american|INT|number of race|
|count_american indian and alaska native|INT|number of race|

|visa_dim| type| description|
|---|---|---|
|i94visa|FLOAT| Visa designation|
|visatype|VARCHAR| Visa Type|	

#### Step 5: Complete Project Write Up


**SCENARIOS**
- The size of the data is increased by 100x : In this case spark would have tobe used. Additionally an Amazon EMR cluster should be used to run the ETL to del with the increased size. This offers basically unlimited computing power as it scales in and out depending on the need of the model. The data can also be uploaded directly to S3.
- The Pipelines have to be run at 7am on a daily basis: This can be easily set up using Airflow. A DAG can be set up with each step of the ETL included within it. A scheduler is then set up to ensure it is ran at the times needed without human intervention.
- The Database needs too be accessed by 100+ people : Cloud capabilities on AWS allows large amounts of people to access the same data remotely and allows the root user to have control over their permissions. Due to the large size of the data and number of people who require access, redshift may be the most ideal solution in this scenario.